# Loading libraries

In [1]:
import numpy as np
import pandas as pd
import pickle
import xml.etree.ElementTree as ET

# Loading MyPersonality data sets

In [2]:
big_five = pd.read_csv('Datasets\\big5.csv')
country = pd.read_csv('Datasets\\country.csv')
demog = pd.read_csv('Datasets\\demog.csv')

C:\ProgramData\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Preprocessing

In [73]:
big_five = big_five[big_five['blocks'] == 20]
country = country[country['country'] == 'united states']
demog = demog[pd.notnull(demog['gender']) &
              pd.notnull(demog['age']) &
              pd.notnull(demog['relationship_status'])]

# Join data sets

In [10]:
user_data = pd.merge(left=big_five, right=country, how='inner', on='userid')
user_data = pd.merge(left=user_data, right=demog, how='inner', on='userid')

del(big_five)
del(country)
del(demog)

# Importing Friendship DIADS

In [3]:
# Gephi works up to 100,000 nodes and 1,000,000 edges
# Temporary solution
# user_data = user_data[:100000]
user_id = big_five['userid']

n = 0
n_max = 200
debug = False

chunksize = 10 ** 7
user_diads = pd.DataFrame({'friend1':[], 'friend2': []})
for chunk in pd.read_csv('Datasets\\fb_friendship.csv', chunksize=chunksize):
    %time temp = chunk[chunk['friend1'].isin(user_id) & chunk['friend2'].isin(user_id)]
    user_diads = pd.concat([user_diads, temp], ignore_index=True)
    del(temp)
     
    print user_diads.shape
    
    n+=1
    if debug & n >= n_max:
        break

Wall time: 6.68 s
(12299, 2)
Wall time: 6.34 s
(28553, 2)
Wall time: 7.26 s
(52760, 2)
Wall time: 7.29 s
(96985, 2)
Wall time: 7.34 s
(144481, 2)
Wall time: 7.28 s
(191682, 2)
Wall time: 7.35 s
(235436, 2)
Wall time: 7.58 s
(278698, 2)
Wall time: 7.58 s
(320582, 2)
Wall time: 8.34 s
(363416, 2)
Wall time: 8.96 s
(407945, 2)
Wall time: 8.86 s
(450260, 2)
Wall time: 8.02 s
(495961, 2)
Wall time: 7.92 s
(539325, 2)
Wall time: 8.2 s
(582404, 2)
Wall time: 7.46 s
(625142, 2)
Wall time: 7.42 s
(665986, 2)
Wall time: 7.49 s
(703544, 2)
Wall time: 7.97 s
(842056, 2)
Wall time: 8.67 s
(991440, 2)
Wall time: 8.63 s
(1112380, 2)
Wall time: 8.65 s
(1182399, 2)
Wall time: 5.16 s
(1199090, 2)


# Saving results

In [4]:
user_data.to_csv('Datasets\\data_filtered.csv', index=False)
user_diads.to_csv('Datasets\\fb_friendship_filtered_all.csv', index=False)

# Converting to GraphML

In [78]:
graphml = ET.Element('graphml')
graphml.set('xmlns', 'http://graphml.graphdrawing.org/xmlns')
graphml.set('xmlns:xsi', 'http://www.w3.org/2001/XMLSchema-instance')
graphml.set('xsi:schemaLocation', 'http://graphml.graphdrawing.org/xmlns/1.0/graphml.xsd')

graph = ET.SubElement(graphml, 'graph', id='friendship_network', edgedefault='undirected')
for user in user_data.itertuples():
    ET.SubElement(graph, 'node', id=user.userid)
for dyad in user_diads.itertuples():
    ET.SubElement(graph, 'edge', id=dyad.friend1+dyad.friend2, source=dyad.friend1, target=dyad.friend2)

xml = ET.ElementTree(graphml)
xml.write('friendship_network.graphml', encoding='utf-8', xml_declaration=True)

## GraphML example

In [47]:
#<?xml version="1.0" encoding="UTF-8"?>
#<graphml xmlns="http://graphml.graphdrawing.org/xmlns"  
#    xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
#    xsi:schemaLocation="http://graphml.graphdrawing.org/xmlns/1.0/graphml.xsd">
#  <graph id="G" edgedefault="undirected">
#    <node id="n0"/>
#    <node id="n1"/>
#    <edge id="e1" source="n0" target="n1"/>
#  </graph>
#</graphml>